# October 17th Meeting

## Implementation of 4th Order Operators

In [3]:
function Dx_p4(y_in, Nx, Ny, h)
	d  = [1/12 -2/3 0 2/3 -1/12]
    bd = [-24/17  59/34  -4/17  -3/34  0     0;
           -1/2    0      1/2    0     0     0;
            4/43 -59/86   0     59/86 -4/43  0;
            3/98   0    -59/98   0    32/49 -4/49]
	N = Nx*Ny
	y_out = zeros(N)

	for row_num = 1:size(bd)[1]
		idx = (row_num - 1)*Ny+1:(row_num)*Ny
		y_out[idx] = (bd[row_num,1] * y_in[idx .+ (1 - row_num)*Ny] + bd[row_num,2] * y_in[idx .+ Ny*(2-row_num)] + bd[row_num,3] * y_in[idx .+ Ny*(3-row_num)] + bd[row_num,4] * y_in[idx .+ Ny*(4-row_num)] + bd[row_num,5] * y_in[idx .+ Ny*(5-row_num)] + bd[row_num,6] * y_in[idx .+ Ny*(6-row_num)]) ./ h
	end

	idx1 = (size(bd)[1])*Ny+1:N-(size(bd)[1])*Ny
	y_out[idx1] = (d[1]*y_in[idx1 .- 2*Ny] + d[2]*y_in[idx1 .- Ny] + d[3]*y_in[idx1] + d[4]*y_in[idx1 .+ Ny] + d[5] * y_in[idx1 .+ 2*Ny]) ./ (h)

	for row_num = 1:size(bd)[1]
		idx2 = N-(row_num)*Ny+1:N .- (row_num - 1)*Ny
		y_out[idx2] = -(bd[row_num,1] * y_in[idx2 .+ (row_num - 1)*Ny] + bd[row_num,2] * y_in[idx2 .- Ny*(2-row_num)] + bd[row_num,3] * y_in[idx2 .- Ny*(3-row_num)] + bd[row_num,4]*y_in[idx2 .- Ny*(4-row_num)] + bd[row_num,5] * y_in[idx2 .- Ny*(5-row_num)] + bd[row_num,6] * y_in[idx2 .- Ny*(6-row_num)]) ./ h
	end

	return y_out
end

Dx_p4 (generic function with 1 method)

```jl
julia> y_out_1 ≈ y_out_2
true

julia> reshape(y_out_1 - y_out_2,11,11)
11×11 Array{Float64,2}:
  0.0           4.44089e-16   2.22045e-16   8.88178e-16   0.0          …  -8.88178e-16  -8.88178e-16   0.0          -3.33067e-16
 -4.44089e-16  -4.44089e-16   2.22045e-16  -1.77636e-15   8.88178e-16      3.33067e-16   0.0           8.88178e-16   0.0
 -3.55271e-15  -8.88178e-16   8.88178e-16   0.0           2.22045e-16      1.77636e-15  -1.77636e-15   0.0           0.0
  0.0           0.0           1.77636e-15  -1.77636e-15  -1.77636e-15      1.77636e-15  -1.77636e-15  -1.77636e-15   3.55271e-15
  0.0           0.0          -1.77636e-15   0.0          -3.19189e-16     -4.44089e-16   0.0           0.0          -3.55271e-15
  3.55271e-15   0.0           0.0           0.0           0.0          …   0.0           1.77636e-15   8.88178e-16   4.44089e-16
  0.0           1.77636e-15   1.11022e-16   4.44089e-16  -4.44089e-16      8.88178e-16   3.55271e-15   0.0           0.0
  8.88178e-16  -1.77636e-15   0.0          -1.08247e-15   0.0              0.0           0.0           8.88178e-16  -3.55271e-15
 -3.55271e-15   0.0           4.44089e-16  -4.44089e-16  -8.88178e-16      0.0           2.66454e-15  -1.77636e-15   0.0
  7.10543e-15   0.0           7.10543e-15  -1.77636e-15  -7.77156e-16      2.77556e-16   8.88178e-16   8.88178e-16   0.0
  0.0           0.0           0.0           8.88178e-16   0.0          …   1.77636e-15   0.0           0.0           1.77636e-15
```